In [1]:
!pip install openai
!pip install llama-index


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["D:\\University\\Test\\1.pdf"]).load_data()

In [6]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)


from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=5,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "1"}
        ]
    )
)

response = query_engine.query(
    "WHat is TRIZ and why should we integrate it with the six sigma?", 
)

print(str(response))

TRIZ is a methodology that focuses on systematic innovation and technical creativity. It is effective in analyzing customer needs and developing innovative solutions to meet those needs. Integrating TRIZ with Six Sigma can enhance the traditional techniques of Six Sigma by providing a structured approach to problem-solving and innovation.


In [8]:
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=5,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    ''' please consider all pages in this paper and give me a comprehensive definition for the integration of TRIZ and Six Sigma''', 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "integration of TRIZ and Six Sigma", "page_numbers": []}
=== Function Output ===
The integration of TRIZ and Six Sigma has been found to be effective in improving banking services. This combination enhances traditional Lean Six Sigma techniques by providing innovative solutions to meet customer needs and improve processes.


In [21]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of the paper"
    ),
)

In [10]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    
    ''' please consider all pages in this paper and give me a comprehensive definition for the integration of TRIZ and Six Sigma''', 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "integration of TRIZ and Six Sigma", "page_numbers": []}
=== Function Output ===
The integration of TRIZ and Six Sigma has been shown to be effective in improving banking services. This combination enhances traditional Lean Six Sigma techniques by providing innovative solutions to meet customer needs and improve service quality and speed.


In [11]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    
    '''give me a comprehensive definition of TRIZ''', 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "TRIZ"}
=== Function Output ===
TRIZ is a problem-solving methodology that allows for creative solutions by revealing and solving problems in various fields. It focuses on developing inventive thinking skills and a creative personality. TRIZ is effective in analyzing customer needs and developing innovative solutions to meet those needs. It has been successfully applied in various industries, including manufacturing, to improve processes and solve technical problems. TRIZ provides a systematic approach to innovation by drawing principles from innovative inventions across different fields and guiding individuals to understand problems as systems and promote product performance by resolving contradictions.


Agent worker is a list of agents who are expert in executing various task. Agent runner some how manage the agents and specify what should be done in every stage!

What Agent runner does is mapping from task_id to task_state

Task State:
Tasks
Completed States
Step Ques

In [13]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [43]:
response = agent.query(
    "Tell me about TRIZ role in design process, "
    "and how TRIZ and LSS can be related together."
)

Added user message to memory: Tell me about TRIZ role in design process, and how TRIZ and LSS can be related together.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "TRIZ role in design process"}
=== Function Output ===
TRIZ methodology plays a crucial role in the design process by providing a systematic approach to problem-solving and innovation. It allows for the identification and resolution of contradictions in any field of knowledge, fostering creative thinking and innovative solutions. By utilizing TRIZ principles, designers can develop high-level products and service applications effectively. TRIZ has been successfully integrated with problem-solving tools in various industries, enabling the development of inventive solutions and enhancing the overall design process.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Relationship between TRIZ and Lean Six Sigma"}
=== Function Output ===
TRIZ methodology is integrated into

Note: As is clear the agentrunner first breaks down the requiry to the separate tasks then would call related tools and execute the tools after doing each task all of the tasks are integrated in a an overall format.

We can also see the source nodes for each node:

In [18]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: 1.pdf
file_path: D:\University\Test\1.pdf
file_type: application/pdf
file_size: 666613
creation_date: 2024-06-06
last_modified_date: 2024-06-05

Full Terms & Conditions of access and use can be found at
https://www.tandfonline.com/action/journalInformation?journalCode=ctqm20
Total Quality Management
ISSN: 1478-3363 (Print) 1478-3371 (Online) Journal homepage: www.tandfonline.com/journals/ctqm20
Applying Lean Six Sigma and TRIZ methodology in
banking services
Fu-Kwun Wang & Kao-Shan Chen
To cite this article:  Fu-Kwun Wang & Kao-Shan Chen (2010) Applying Lean Six Sigma and
TRIZ methodology in banking services, Total Quality Management, 21:3, 301-315, DOI:
10.1080/14783360903553248
To link to this article:  https://doi.org/10.1080/14783360903553248
Published online: 08 Apr 2010.
Submit your article to this journal 
Article views: 2569
View related articles 
Citing articles: 19 View citing articles


Note:With the agent runner We can also use the history of the converstion to the agent so that we can ask various questions to the paper with regards to having the history.

In [19]:
response = agent.chat(
    "Please list all the usefull tools in LSS"
)

Added user message to memory: Please list all the usefull tools in LSS
=== LLM Response ===
Some useful tools in Lean Six Sigma (LSS) include:

1. Value Stream Mapping
2. 5S (Sort, Set in order, Shine, Standardize, Sustain)
3. Kaizen Events
4. Fishbone Diagram (Ishikawa)
5. Control Charts
6. Pareto Analysis
7. Root Cause Analysis
8. Failure Mode and Effects Analysis (FMEA)
9. Process Mapping
10. Poka-Yoke (Error Proofing)

These tools are commonly used in Lean Six Sigma projects to identify opportunities for improvement, analyze processes, and implement effective solutions to enhance business performance.


In [20]:
response = agent.chat("Please describe the second tool")

Added user message to memory: Please describe the second tool
=== Calling Function ===
Calling function: summary_tool with args: {"input": "5S (Sort, Set in order, Shine, Standardize, Sustain)"}
=== Function Output ===
The application of Lean Six Sigma and TRIZ methodology in the banking services industry has shown effective results in improving operational efficiency and customer satisfaction. By integrating TRIZ into the Lean Six Sigma approach, the company was able to identify and address key issues such as reducing waiting times, improving operational processes, and enhancing customer service. The combination of Lean Six Sigma and TRIZ methodologies allowed for a systematic approach to problem-solving and innovation, leading to significant improvements in process capability and cost savings. The control phase involved implementing a robust control plan to prevent system failures and ensure sustained gains. Through the implementation of DMAIC and TRIZ, the company successfully achie

**Lower-Level: Debuggability and Control**

Note: with agent runner you can also control the tasks of the agents in each step and debugg and modify them!
Note: With this option you can also pause the excution for a while and continue

In [23]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [40]:
task = agent.create_task(
     "Tell me about TRIZ role in design process, "
    "and how TRIZ and LSS can be related together."
)

In [41]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about TRIZ role in design process, and how TRIZ and LSS can be related together.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "TRIZ role in design process"}
=== Function Output ===
TRIZ methodology plays a crucial role in the design process by providing a systematic approach to problem-solving and innovation. It allows for the identification of contradictions, creative problem-solving, and the development of inventive solutions to meet customer needs. TRIZ helps in analyzing customer requirements, developing innovative solutions, and enhancing the overall performance of products or services by addressing contradictions effectively. Through the application of TRIZ principles, designers can develop high-level products and improve service applications by identifying and solving complex problems in a structured and innovative manner.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "TRIZ and L

If we compare it with the previous response of the same query we can vividly see that the llm did not do the last step.

In [45]:
#task

In [46]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 8f4a2e59-53e2-4a4f-bf51-b611f38cede4: 1
TRIZ methodology plays a crucial role in the design process by providing a systematic approach to problem-solving and innovation. It allows for the identification of contradictions, creative problem-solving, and the development of inventive solutions to meet customer needs. TRIZ helps in analyzing customer requirements, developing innovative solutions, and enhancing the overall performance of products or services by addressing contradictions effectively. Through the application of TRIZ principles, designers can develop high-level products and improve service applications by identifying and solving complex problems in a structured and innovative manner.


In [47]:
#completed_steps

In [48]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 8f4a2e59-53e2-4a4f-bf51-b611f38cede4: 1


TaskStep(task_id='8f4a2e59-53e2-4a4f-bf51-b611f38cede4', step_id='260c613a-bead-4821-9003-80a6c504bdab', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [49]:
step_output = agent.run_step(
    task.task_id, input="How Lss help TRIZ?"
)

Added user message to memory: How Lss help TRIZ?
=== Calling Function ===
Calling function: summary_tool with args: {"input": "How Lean Six Sigma helps TRIZ"}
=== Function Output ===
Lean Six Sigma helps TRIZ by providing a structured methodology for process improvement and waste reduction, which complements TRIZ's innovative problem-solving approach. By integrating Lean Six Sigma with TRIZ, organizations can effectively identify and eliminate process variations, reduce defects, enhance customer satisfaction, and improve operational efficiency in a systematic and innovative manner. This integration allows for a comprehensive improvement strategy that combines the strengths of both methodologies to drive continuous enhancement in service quality and operational performance.


In [50]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Lean Six Sigma helps TRIZ by providing a structured methodology for process improvement and waste reduction, which complements TRIZ's innovative problem-solving approach. By integrating Lean Six Sigma with TRIZ, organizations can effectively identify and eliminate process variations, reduce defects, enhance customer satisfaction, and improve operational efficiency in a systematic and innovative manner. This integration allows for a comprehensive improvement strategy that combines the strengths of both methodologies to drive continuous enhancement in service quality and operational performance.
True


Note: Step out put is last only return the last part of the LLM

In [51]:
response = agent.finalize_response(task.task_id)

In [52]:
print(str(response))

Lean Six Sigma helps TRIZ by providing a structured methodology for process improvement and waste reduction, which complements TRIZ's innovative problem-solving approach. By integrating Lean Six Sigma with TRIZ, organizations can effectively identify and eliminate process variations, reduce defects, enhance customer satisfaction, and improve operational efficiency in a systematic and innovative manner. This integration allows for a comprehensive improvement strategy that combines the strengths of both methodologies to drive continuous enhancement in service quality and operational performance.
